In [1]:
import numpy as np
import pandas as pd

#### File IO

In [2]:
import io
import re

def read_file_segment(file_path_or_line_list, start_pattern, end_pattern):
    """
    To read a segment of file, marked with start_pattern and end_pattern
    and to return as a string.
    
    Args:
        file_path_or_line_list:
                       the input file path or a list of lines
        start_pattern: the pattern of the line after which
                       contents are to be returned
        end_pattern:   the pattern of the line before which
                       contents are to be returned
    
    Returns:
        a list of strings each containing a line of texts
    """
    results = []
    state_copy = False
    empty_or_comment_c = re.compile("^((\s*)|(##.*))$")
    start_pattern_c = re.compile(re.escape(start_pattern)) if start_pattern else None
    end_pattern_c = re.compile(re.escape(end_pattern)) if end_pattern else None
    with (io.open(file_path_or_line_list, mode="r", encoding="utf-8") if not isinstance(file_path_or_line_list, (list))
          else io.StringIO("".join(file_path_or_line_list))) as f:
        line = f.readline()
        while line:
            if empty_or_comment_c.match(line):
                line = f.readline()
                continue
            if state_copy and end_pattern_c and re.search(end_pattern_c, line):
                break
            if state_copy:
                results.append(line)
            else:
                if not start_pattern_c or re.search(start_pattern_c, line):
                    state_copy = True
            line = f.readline()
    return results

#### Padas dataframe construction

In [3]:
import io

def read_dataframe(text, sep='[,_\s]+'):
    with io.StringIO(text) as f:
        return pd.read_csv(f, sep=sep, engine='python', header=None)

#### t-Statistics

In [4]:
from scipy import stats

def calc_p_value(df1, df2):
    return (stats.ttest_ind(df1, df2, equal_var=False)[1] / 2.0)

def calc_gain(df1, df2):
    m1 = df1.mean()
    m2 = df2.mean()
    gain = (m2 - m1) / m1
    return [ itm * 100 for itm in gain.tolist()]

#### ListNet MQ2007 Three-layer NN results

In [5]:
listnet_mq2007_three_layer = read_file_segment(
    'resources/mq2007-threelayer-multi.txt', 
    'ListNet (three_layer) MQ2007 metrics average of [10] runs',
    'ListMLE (three_layer) MQ2007 metrics average of [10] runs')
listnet_mq2007_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2007_three_layer,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
listnet_mq2007_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2007_three_layer,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(listnet_mq2007_three_layer_ndcgs)
print(listnet_mq2007_three_layer_errs)

        1       2       3       4       5       6
0  0.4732  0.4762  0.4848  0.5178  0.5868  0.6972
1  0.4742  0.4713  0.4775  0.5161  0.5845  0.6953
2  0.4757  0.4773  0.4851  0.5174  0.5870  0.6969
3  0.4762  0.4738  0.4809  0.5170  0.5874  0.6962
4  0.4690  0.4739  0.4815  0.5167  0.5867  0.6968
5  0.4756  0.4779  0.4836  0.5168  0.5874  0.6969
6  0.4701  0.4716  0.4815  0.5154  0.5859  0.6952
7  0.4663  0.4737  0.4793  0.5163  0.5871  0.6958
8  0.4760  0.4733  0.4798  0.5163  0.5867  0.6961
9  0.4722  0.4737  0.4829  0.5169  0.5873  0.6970
        1       2       3       4       5       6
0  0.2245  0.3166  0.3433  0.3623  0.3697  0.3723
1  0.2241  0.3149  0.3409  0.3610  0.3685  0.3710
2  0.2239  0.3169  0.3432  0.3623  0.3700  0.3725
3  0.2239  0.3156  0.3417  0.3618  0.3693  0.3718
4  0.2203  0.3132  0.3388  0.3588  0.3663  0.3690
5  0.2255  0.3185  0.3445  0.3638  0.3714  0.3739
6  0.2228  0.3143  0.3418  0.3606  0.3682  0.3707
7  0.2203  0.3145  0.3398  0.3599  0.3676  0.3701


#### Alpha-divergence MQ2007 Three-layer Results

In [6]:
multi_mq2007_three_layer_aggregate = read_file_segment(
    'resources/mq2007-threelayer-alpha.txt', 
    'Dataset (three_layer) MQ2007 NDCGs',
    None)
alpha_mq2007_three_layer_ndcgs = read_file_segment(
    multi_mq2007_three_layer_aggregate,
    "ObjFunc", "Dataset"
    )
alpha_mq2007_three_layer_errs = read_file_segment(
    multi_mq2007_three_layer_aggregate,
    "Dataset (three_layer) MQ2007 ERRs", None
    )
alpha_mq2007_three_layer_errs = read_file_segment(
    alpha_mq2007_three_layer_errs,
    "ObjFunc", None
    )
alpha_mq2007_three_layer_ndcgs = read_dataframe("".join(alpha_mq2007_three_layer_ndcgs))
alpha_mq2007_three_layer_errs = read_dataframe("".join(alpha_mq2007_three_layer_errs))

max_alpha_loc = alpha_mq2007_three_layer_ndcgs[1].idxmax()
max_alpha_mq2007_three_layer_val = alpha_mq2007_three_layer_ndcgs.loc[max_alpha_loc, 0]
max_alpha_ndcgs = alpha_mq2007_three_layer_ndcgs.loc[max_alpha_loc]
max_alpha_errs = alpha_mq2007_three_layer_errs.loc[max_alpha_loc]
print(alpha_mq2007_three_layer_ndcgs)
print(max_alpha_ndcgs)
print(alpha_mq2007_three_layer_errs)
print(max_alpha_errs)
print('max_alpha:', max_alpha_mq2007_three_layer_val)

max_alpha_mq2007_three_layer_ndcg_err = read_file_segment(
    'resources/mq2007-threelayer-alpha.txt', 
    'Alpha[0.4] Divergence (three_layer) MQ2007 [10] runs',
    'Alpha[0.5] Divergence (three_layer) MQ2007 [10] runs')
max_alpha_mq2007_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2007_three_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_mq2007_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2007_three_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_mq2007_three_layer_ndcgs)
print(max_alpha_mq2007_three_layer_errs)

           0       1       2       3       4       5       6
0   Alph-0.9  0.4696  0.4727  0.4821  0.5159  0.5855  0.6961
1   Alph-0.6  0.4676  0.4718  0.4804  0.5156  0.5854  0.6957
2   Alph-0.4  0.4701  0.4738  0.4809  0.5157  0.5859  0.6959
3   Alph-0.2  0.4722  0.4742  0.4820  0.5162  0.5866  0.6965
4   Alph-0.1  0.4740  0.4741  0.4826  0.5170  0.5866  0.6964
5    Alph0.1  0.4737  0.4744  0.4819  0.5161  0.5868  0.6964
6    Alph0.2  0.4724  0.4734  0.4815  0.5162  0.5863  0.6961
7    Alph0.3  0.4755  0.4745  0.4828  0.5173  0.5872  0.6968
8    Alph0.4  0.4771  0.4754  0.4835  0.5178  0.5879  0.6974
9    Alph0.5  0.4744  0.4756  0.4828  0.5171  0.5875  0.6968
10   Alph0.6  0.4735  0.4747  0.4829  0.5173  0.5870  0.6966
11   Alph0.7  0.4755  0.4749  0.4825  0.5168  0.5870  0.6965
0    Alph0.4
1     0.4771
2     0.4754
3     0.4835
4     0.5178
5     0.5879
6     0.6974
Name: 8, dtype: object
           0       1       2       3       4       5       6
0   Alph-0.9  0.2221  0.3145  0.

In [7]:
max_alpha_mq2007_three_layer_ndcg_gains = calc_gain(listnet_mq2007_three_layer_ndcgs, max_alpha_mq2007_three_layer_ndcgs)
print(max_alpha_mq2007_three_layer_ndcg_gains)
max_alpha_mq2007_three_layer_ndcg_p_values = calc_p_value(listnet_mq2007_three_layer_ndcgs, max_alpha_mq2007_three_layer_ndcgs)
print(max_alpha_mq2007_three_layer_ndcg_p_values)
max_alpha_mq2007_three_layer_err_gains = calc_gain(listnet_mq2007_three_layer_errs, max_alpha_mq2007_three_layer_errs)
print(max_alpha_mq2007_three_layer_err_gains)
max_alpha_mq2007_three_layer_err_p_values = calc_p_value(listnet_mq2007_three_layer_errs, max_alpha_mq2007_three_layer_errs)
print(max_alpha_mq2007_three_layer_err_p_values)

[0.9051496246166992, 0.23615240263983686, 0.3716082957919138, 0.21290185224611258, 0.19942728574348761, 0.15366056811326423]
[0.00297988 0.13675461 0.02941939 0.00528577 0.00165266 0.00093601]
[1.3177356460938543, 0.5705953211183745, 0.547247666149654, 0.5203864145929618, 0.496150092180891, 0.49009047824213386]
[0.00127736 0.0043684  0.00451727 0.00176871 0.00240378 0.00215134]


#### Weighted-KL Divergence MQ2007 Three-layer Results

In [8]:
weighted_KL_mq2007_three_layer_aggregate = read_file_segment(
    'resources/mq2007-threelayer-weighted-kl.txt',
    "Dataset (three_layer) MQ2007 NDCGs", None
    )
weighted_KL_mq2007_three_layer_ndcgs = read_file_segment(
    weighted_KL_mq2007_three_layer_aggregate,
    "ObjFunc", "Dataset"
    )
weighted_KL_mq2007_three_layer_errs = read_file_segment(
    weighted_KL_mq2007_three_layer_aggregate,
    "Dataset (three_layer) MQ2007 ERRs", None
    )
weighted_KL_mq2007_three_layer_errs = read_file_segment(
    weighted_KL_mq2007_three_layer_errs,
    "ObjFunc", None
    )
weighted_KL_mq2007_three_layer_ndcgs = read_dataframe("".join(weighted_KL_mq2007_three_layer_ndcgs))
weighted_KL_mq2007_three_layer_errs = read_dataframe("".join(weighted_KL_mq2007_three_layer_errs))

max_lambda_loc = weighted_KL_mq2007_three_layer_ndcgs[1].idxmax()
max_lambda_mq2007_three_layer_val = weighted_KL_mq2007_three_layer_ndcgs.loc[max_lambda_loc, 0]
max_lambda_ndcgs = weighted_KL_mq2007_three_layer_ndcgs.loc[max_lambda_loc]
max_lambda_errs = weighted_KL_mq2007_three_layer_errs.loc[max_lambda_loc]
print(weighted_KL_mq2007_three_layer_ndcgs)
print(max_lambda_ndcgs)
print(weighted_KL_mq2007_three_layer_errs)
print(max_lambda_errs)
print('max_lambdaa:', max_lambda_mq2007_three_layer_val)

max_weighted_KL_mq2007_three_layer_ndcg_err = read_file_segment(
    'resources/mq2007-threelayer-weighted-kl.txt', 
    'Lambda[-0.9] KL Divergence (three_layer) MQ2007 [10] runs',
    'Lambda[-0.6] KL Divergence (three_layer) MQ2007 [10] runs')
max_weighted_KL_mq2007_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2007_three_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_weighted_KL_mq2007_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2007_three_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_weighted_KL_mq2007_three_layer_ndcgs)
print(max_weighted_KL_mq2007_three_layer_errs)

          0       1       2       3       4       5       6
0   Wgt-0.9  0.4754  0.4740  0.4819  0.5166  0.5872  0.6967
1   Wgt-0.6  0.4736  0.4746  0.4820  0.5165  0.5868  0.6961
2   Wgt-0.4  0.4738  0.4748  0.4828  0.5167  0.5867  0.6961
3   Wgt-0.2  0.4739  0.4751  0.4830  0.5166  0.5867  0.6962
4   Wgt-0.1  0.4745  0.4750  0.4826  0.5174  0.5869  0.6966
5    Wgt0.1  0.4719  0.4732  0.4817  0.5157  0.5861  0.6958
6    Wgt0.2  0.4712  0.4742  0.4809  0.5150  0.5858  0.6955
7    Wgt0.3  0.4712  0.4734  0.4813  0.5147  0.5855  0.6954
8    Wgt0.4  0.4702  0.4728  0.4812  0.5148  0.5851  0.6953
9    Wgt0.5  0.4707  0.4726  0.4816  0.5153  0.5858  0.6956
10   Wgt0.6  0.4735  0.4725  0.4818  0.5153  0.5855  0.6956
11   Wgt0.7  0.4725  0.4717  0.4816  0.5148  0.5850  0.6953
0    Wgt-0.9
1     0.4754
2      0.474
3     0.4819
4     0.5166
5     0.5872
6     0.6967
Name: 0, dtype: object
          0       1       2       3       4       5       6
0   Wgt-0.9  0.2264  0.3168  0.3427  0.3624  0

In [9]:
max_weighted_KL_mq2007_three_layer_ndcg_gains = calc_gain(listnet_mq2007_three_layer_ndcgs, max_weighted_KL_mq2007_three_layer_ndcgs)
print(max_weighted_KL_mq2007_three_layer_ndcg_gains)
max_weighted_KL_mq2007_three_layer_ndcg_p_values = calc_p_value(listnet_mq2007_three_layer_ndcgs, max_weighted_KL_mq2007_three_layer_ndcgs)
print(max_weighted_KL_mq2007_three_layer_ndcg_p_values)
max_weighted_KL_mq2007_three_layer_err_gains = calc_gain(listnet_mq2007_three_layer_errs, max_weighted_KL_mq2007_three_layer_errs)
print(max_weighted_KL_mq2007_three_layer_err_gains)
max_weighted_KL_mq2007_three_layer_err_p_values = calc_p_value(listnet_mq2007_three_layer_errs, max_weighted_KL_mq2007_three_layer_errs)
print(max_weighted_KL_mq2007_three_layer_err_p_values)

[0.5371682351697246, -0.06325510784996362, 0.037368431979099524, -0.01741924245648547, 0.09545237608236258, 0.05744320303297884]
[0.04779298 0.38922048 0.42701717 0.41786357 0.11425434 0.12857103]
[1.479091031329836, 0.41843656882013186, 0.27508706212870454, 0.31001743848090196, 0.3253443227415775, 0.3339077983627798]
[0.00051375 0.04360473 0.10912917 0.04767983 0.03559328 0.0296189 ]


#### Alpha Divergence + Entropy Regularization MQ2007 Three-Layer Network

In [10]:
# Manually composed file from multiple parallel runs.
# The first part of the file contains NDCGs
# and the second part of the file contains ERRs
file_path = 'resources/mq2007-threelayer-alpha-entropy-secondrun.txt'
alpha_entropy_ndcg_data = read_file_segment(file_path, "Dataset (three_layer) MQ2007 NDCG", "Dataset (three_layer) MQ2007 ERRs")
alpha_entropy_err_data = read_file_segment(file_path, "Dataset (three_layer) MQ2007 ERRs", None)
alpha_entropy_ndcg_df = read_dataframe("".join(alpha_entropy_ndcg_data))
alpha_entropy_ndcg_df.columns = ['alpha', 'lambda', 'nDCG@1', 'nDCG@3', 'nDCG@5', 'nDCG@10', 'nDCG@20', 'nDCG@50']
alpha_entropy_err_df = read_dataframe("".join(alpha_entropy_err_data))
alpha_entropy_err_df.columns = ['alpha', 'lambda', 'ERR@1', 'ERR@3', 'ERR@5', 'ERR@10', 'ERR@20', 'ERR@50']

In [11]:
ndcg_df_sorted = alpha_entropy_ndcg_df.sort_values(by='nDCG@1', ascending=False)
print(ndcg_df_sorted[:5])
err_df_sorted = alpha_entropy_err_df.sort_values(by='ERR@1', ascending=False)
print(err_df_sorted[:5])

best_alpha_mq2007_three_layer = 0.4
best_lambda_mq2007_three_layer = -0.4

    alpha  lambda  nDCG@1  nDCG@3  nDCG@5  nDCG@10  nDCG@20  nDCG@50
24   -0.4    -0.9  0.4785  0.4740  0.4818   0.5172   0.5875   0.6975
98    0.4    -0.4  0.4773  0.4756  0.4831   0.5169   0.5876   0.6969
48   -0.1    -0.9  0.4765  0.4720  0.4810   0.5159   0.5862   0.6968
0    -0.9    -0.9  0.4764  0.4744  0.4810   0.5158   0.5875   0.6971
53   -0.1     0.1  0.4762  0.4746  0.4823   0.5167   0.5867   0.6967
    alpha  lambda   ERR@1   ERR@3   ERR@5  ERR@10  ERR@20  ERR@50
24   -0.4    -0.9  0.2269  0.3169  0.3430  0.3628  0.3704  0.3730
0    -0.9    -0.9  0.2268  0.3177  0.3434  0.3631  0.3708  0.3732
48   -0.1    -0.9  0.2265  0.3159  0.3422  0.3620  0.3696  0.3722
98    0.4    -0.4  0.2262  0.3171  0.3434  0.3628  0.3704  0.3729
36   -0.2    -0.9  0.2262  0.3162  0.3425  0.3621  0.3698  0.3724


In [12]:
max_alpha_lambda_aggregate = read_file_segment(
    'resources/mq2007-alpha-and-entropy-0.4.txt',
    'Entropy regulated[-0.4] alpha-Divergence[0.4] (three_layer) MQ2007 [10]',
    'Entropy regulated[-0.2] alpha-Divergence[0.4] (three_layer) MQ2007 [10] runs')

max_alpha_lambda_mq2007_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_lambda_mq2007_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_lambda_mq2007_three_layer_ndcgs)
print(max_alpha_lambda_mq2007_three_layer_errs)

        1       2       3       4       5       6
0  0.4805  0.4762  0.4834  0.5174  0.5882  0.6974
1  0.4769  0.4761  0.4834  0.5174  0.5878  0.6971
2  0.4797  0.4731  0.4820  0.5166  0.5859  0.6958
3  0.4786  0.4769  0.4839  0.5185  0.5894  0.6979
4  0.4800  0.4819  0.4875  0.5202  0.5900  0.6993
5  0.4747  0.4753  0.4795  0.5164  0.5873  0.6966
6  0.4731  0.4748  0.4811  0.5144  0.5862  0.6956
7  0.4755  0.4758  0.4855  0.5179  0.5871  0.6970
8  0.4767  0.4730  0.4833  0.5153  0.5868  0.6956
9  0.4770  0.4729  0.4816  0.5152  0.5870  0.6966
        1       2       3       4       5       6
0  0.2272  0.3177  0.3439  0.3631  0.3709  0.3733
1  0.2265  0.3172  0.3438  0.3632  0.3707  0.3733
2  0.2259  0.3160  0.3425  0.3624  0.3697  0.3723
3  0.2269  0.3181  0.3444  0.3641  0.3716  0.3741
4  0.2279  0.3203  0.3461  0.3653  0.3728  0.3753
5  0.2269  0.3180  0.3426  0.3628  0.3704  0.3730
6  0.2227  0.3154  0.3413  0.3606  0.3685  0.3710
7  0.2259  0.3172  0.3442  0.3630  0.3705  0.3730


In [13]:
max_alpha_lambda_mq2007_three_layer_ndcg_gains = calc_gain(listnet_mq2007_three_layer_ndcgs, max_alpha_lambda_mq2007_three_layer_ndcgs)
print(max_alpha_lambda_mq2007_three_layer_ndcg_gains)
max_alpha_lambda_mq2007_three_layer_ndcg_p_values = calc_p_value(listnet_mq2007_three_layer_ndcgs, max_alpha_lambda_mq2007_three_layer_ndcgs)
print(max_alpha_lambda_mq2007_three_layer_ndcg_p_values)
max_alpha_lambda_mq2007_three_layer_err_gains = calc_gain(listnet_mq2007_three_layer_errs, max_alpha_lambda_mq2007_three_layer_errs)
print(max_alpha_lambda_mq2007_three_layer_err_gains)
max_alpha_lambda_mq2007_three_layer_err_p_values = calc_p_value(listnet_mq2007_three_layer_errs, max_alpha_lambda_mq2007_three_layer_errs)
print(max_alpha_lambda_mq2007_three_layer_err_p_values)

[0.9347573226181858, 0.28043097813481727, 0.29687143183376086, 0.05032225598545261, 0.1517010977023158, 0.07898440417036784]
[0.00204067 0.11943465 0.09624905 0.33336188 0.04769566 0.10968771]
[1.3625565864371776, 0.5325556330438315, 0.48871850399460043, 0.42627397791125365, 0.4256588222535409, 0.42546316242997917]
[0.00027489 0.01174858 0.0138908  0.01013512 0.00778709 0.00672034]


#### ListNet MQ2008 Three-layer NN results

In [14]:
listnet_mq2008_three_layer = read_file_segment(
    'resources/mq2008-threelayer-multi.txt', 
    'ListNet (three_layer) MQ2008 metrics average of [10] runs',
    'ListMLE (three_layer) MQ2008 metrics average of [10] runs')
listnet_mq2008_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2008_three_layer,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
listnet_mq2008_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2008_three_layer,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(listnet_mq2008_three_layer_ndcgs)
print(listnet_mq2008_three_layer_errs)

        1       2       3       4       5       6
0  0.4651  0.4804  0.5214  0.6037  0.6595  0.6920
1  0.4631  0.4865  0.5311  0.6040  0.6614  0.6934
2  0.4567  0.4714  0.5273  0.5997  0.6591  0.6909
3  0.4647  0.4856  0.5323  0.6061  0.6629  0.6955
4  0.4696  0.4936  0.5375  0.6094  0.6661  0.6987
5  0.4522  0.4796  0.5211  0.6040  0.6613  0.6927
6  0.4567  0.4760  0.5253  0.6036  0.6590  0.6898
7  0.4609  0.4774  0.5288  0.5987  0.6582  0.6896
8  0.4791  0.4841  0.5344  0.6102  0.6661  0.6964
9  0.4836  0.4947  0.5364  0.6088  0.6676  0.6989
        1       2       3       4       5       6
0  0.2850  0.3820  0.4099  0.4255  0.4287  0.4293
1  0.2844  0.3877  0.4139  0.4283  0.4320  0.4326
2  0.2748  0.3724  0.4060  0.4192  0.4226  0.4233
3  0.2788  0.3810  0.4089  0.4232  0.4264  0.4272
4  0.2775  0.3842  0.4127  0.4258  0.4291  0.4298
5  0.2698  0.3740  0.4028  0.4181  0.4214  0.4220
6  0.2776  0.3782  0.4074  0.4223  0.4255  0.4262
7  0.2756  0.3745  0.4038  0.4185  0.4220  0.4227


#### Alpha-divergence MQ2008 Three-layer Results

In [15]:
multi_mq2008_three_layer_aggregate = read_file_segment(
    'resources/mq2008-threelayer-multi.txt', 
    'Dataset (three_layer) MQ2008 NDCG',
    None)
alpha_mq2008_three_layer_ndcgs = read_file_segment(
    multi_mq2008_three_layer_aggregate,
    "ListMLE", "Wgt"
    )
alpha_mq2008_three_layer_errs = read_file_segment(
    multi_mq2008_three_layer_aggregate,
    "Dataset (three_layer) MQ2008 ERRs", "Wgt"
    )
alpha_mq2008_three_layer_errs = read_file_segment(
    alpha_mq2008_three_layer_errs,
    "ListMLE", "Wgt"
    )
alpha_mq2008_three_layer_ndcgs = read_dataframe("".join(alpha_mq2008_three_layer_ndcgs))
alpha_mq2008_three_layer_errs = read_dataframe("".join(alpha_mq2008_three_layer_errs))

max_alpha_loc = alpha_mq2008_three_layer_ndcgs[1].idxmax()
max_alpha_mq2008_three_layer_val = alpha_mq2008_three_layer_ndcgs.loc[max_alpha_loc, 0]
max_alpha_ndcgs = alpha_mq2008_three_layer_ndcgs.loc[max_alpha_loc]
max_alpha_errs = alpha_mq2008_three_layer_errs.loc[max_alpha_loc]
print(alpha_mq2008_three_layer_ndcgs)
print(max_alpha_ndcgs)
print(alpha_mq2008_three_layer_errs)
print(max_alpha_errs)
print('max_alpha:', max_alpha_mq2008_three_layer_val)

max_alpha_mq2008_three_layer_ndcg_err = read_file_segment(
    'resources/mq2008-threelayer-multi.txt', 
    'Alpha[0.5] Divergence (three_layer) MQ2008 [10] runs',
    'Alpha[1.3] Divergence (three_layer) MQ2008 [10] runs')
max_alpha_mq2008_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2008_three_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_mq2008_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2008_three_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_mq2008_three_layer_ndcgs)
print(max_alpha_mq2008_three_layer_errs)

          0       1       2       3       4       5       6
0  Alph-0.5  0.4590  0.4753  0.5206  0.6017  0.6583  0.6903
1   Alph0.5  0.4764  0.4883  0.5341  0.6099  0.6665  0.6977
2   Alph1.3  0.4741  0.4864  0.5298  0.6072  0.6646  0.6960
3   Alph1.4  0.4761  0.4855  0.5311  0.6056  0.6632  0.6952
4   Alph1.5  0.4698  0.4867  0.5311  0.6059  0.6636  0.6948
5   Alph1.6  0.4754  0.4866  0.5310  0.6070  0.6642  0.6952
6   Alph1.7  0.4709  0.4848  0.5278  0.6049  0.6622  0.6936
7   Alph2.0  0.4689  0.4836  0.5304  0.6056  0.6636  0.6945
8   Alph5.0  0.4264  0.4493  0.4934  0.5779  0.6391  0.6709
0    Alph0.5
1     0.4764
2     0.4883
3     0.5341
4     0.6099
5     0.6665
6     0.6977
Name: 1, dtype: object
          0       1       2       3       4       5       6
0  Alph-0.5  0.2753  0.3757  0.4039  0.4193  0.4225  0.4232
1   Alph0.5  0.2865  0.3859  0.4143  0.4284  0.4318  0.4324
2   Alph1.3  0.2842  0.3836  0.4120  0.4267  0.4300  0.4307
3   Alph1.4  0.2854  0.3838  0.4119  0.4264  0

In [16]:
max_alpha_mq2008_three_layer_ndcg_gains = calc_gain(listnet_mq2008_three_layer_ndcgs, max_alpha_mq2008_three_layer_ndcgs)
print(max_alpha_mq2008_three_layer_ndcg_gains)
max_alpha_mq2008_three_layer_ndcg_p_values = calc_p_value(listnet_mq2008_three_layer_ndcgs, max_alpha_mq2008_three_layer_ndcgs)
print(max_alpha_mq2008_three_layer_ndcg_p_values)
max_alpha_mq2008_three_layer_err_gains = calc_gain(listnet_mq2008_three_layer_errs, max_alpha_mq2008_three_layer_errs)
print(max_alpha_mq2008_three_layer_err_gains)
max_alpha_mq2008_three_layer_err_p_values = calc_p_value(listnet_mq2008_three_layer_errs, max_alpha_mq2008_three_layer_errs)
print(max_alpha_mq2008_three_layer_err_p_values)

[2.412021411526958, 1.116103783156992, 0.8629805876576709, 0.8399193148375048, 0.6645321089832598, 0.5650124677495961]
[0.01059239 0.03122626 0.03578682 0.00496727 0.00404305 0.00554353]
[2.167789781438291, 1.2965879265091715, 1.1204413416003474, 1.0162697477010092, 0.9965378497239734, 0.9949783954221488]
[0.02547327 0.02196254 0.01331137 0.01797571 0.0187866  0.01835233]


#### Weighted-KL Divergence MQ2008 Three-layer Results

In [17]:
weighted_KL_mq2008_three_layer_ndcgs = read_file_segment(
    multi_mq2008_three_layer_aggregate,
    "Alph5.0", "Dataset"
    )
weighted_KL_mq2008_three_layer_errs = read_file_segment(
    multi_mq2008_three_layer_aggregate,
    "Dataset (three_layer) MQ2008 ERRs", None
    )
weighted_KL_mq2008_three_layer_errs = read_file_segment(
    weighted_KL_mq2008_three_layer_errs,
    "Alph5.0", None
    )
weighted_KL_mq2008_three_layer_ndcgs = read_dataframe("".join(weighted_KL_mq2008_three_layer_ndcgs))
weighted_KL_mq2008_three_layer_errs = read_dataframe("".join(weighted_KL_mq2008_three_layer_errs))

max_lambda_loc = weighted_KL_mq2008_three_layer_ndcgs[1].idxmax()
max_lambda_mq2008_three_layer_val = weighted_KL_mq2008_three_layer_ndcgs.loc[max_lambda_loc, 0]
max_lambda_ndcgs = weighted_KL_mq2008_three_layer_ndcgs.loc[max_lambda_loc]
max_lambda_errs = weighted_KL_mq2008_three_layer_errs.loc[max_lambda_loc]
print(weighted_KL_mq2008_three_layer_ndcgs)
print(max_lambda_ndcgs)
print(weighted_KL_mq2008_three_layer_errs)
print(max_lambda_errs)
print('max_lambdaa:', max_lambda_mq2008_three_layer_val)

max_weighted_KL_mq2008_three_layer_ndcg_err = read_file_segment(
    'resources/mq2008-threelayer-multi.txt', 
    'Lambda[0.7] KL Divergence (three_layer) MQ2008 [10] runs',
    'Lambda[1.4] KL Divergence (three_layer) MQ2008 [10] run')
max_weighted_KL_mq2008_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2008_three_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_weighted_KL_mq2008_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2008_three_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_weighted_KL_mq2008_three_layer_ndcgs)
print(max_weighted_KL_mq2008_three_layer_errs)

         0       1       2       3       4       5       6
0   Wgt0.1  0.4752  0.4844  0.5313  0.6077  0.6637  0.6954
1   Wgt0.7  0.4768  0.4910  0.5331  0.6100  0.6660  0.6971
2   Wgt1.4  0.4739  0.4889  0.5346  0.6115  0.6672  0.6984
3   Wgt2.0  0.4713  0.4899  0.5363  0.6113  0.6675  0.6982
4   Wgt4.0  0.4605  0.4861  0.5335  0.6103  0.6645  0.6962
5   Wgt5.0  0.4590  0.4862  0.5324  0.6103  0.6637  0.6957
6   Wgt6.0  0.4536  0.4856  0.5309  0.6090  0.6634  0.6950
7   Wgt8.0  0.4502  0.4811  0.5282  0.6071  0.6620  0.6936
8  Wgt10.0  0.4517  0.4839  0.5283  0.6076  0.6626  0.6941
9  Wgt15.0  0.4409  0.4794  0.5223  0.6033  0.6590  0.6905
0    Wgt0.7
1    0.4768
2     0.491
3    0.5331
4      0.61
5     0.666
6    0.6971
Name: 1, dtype: object
         0       1       2       3       4       5       6
0   Wgt0.1  0.2854  0.3834  0.4123  0.4269  0.4302  0.4308
1   Wgt0.7  0.2878  0.3879  0.4156  0.4298  0.4331  0.4338
2   Wgt1.4  0.2875  0.3894  0.4170  0.4314  0.4346  0.4353
3   Wgt2

In [18]:
max_weighted_KL_mq2008_three_layer_ndcg_gains = calc_gain(listnet_mq2008_three_layer_ndcgs, max_weighted_KL_mq2008_three_layer_ndcgs)
print(max_weighted_KL_mq2008_three_layer_ndcg_gains)
max_weighted_KL_mq2008_three_layer_ndcg_p_values = calc_p_value(listnet_mq2008_three_layer_ndcgs, max_weighted_KL_mq2008_three_layer_ndcgs)
print(max_weighted_KL_mq2008_three_layer_ndcg_p_values)
max_weighted_KL_mq2008_three_layer_err_gains = calc_gain(listnet_mq2008_three_layer_errs, max_weighted_KL_mq2008_three_layer_errs)
print(max_weighted_KL_mq2008_three_layer_err_gains)
max_weighted_KL_mq2008_three_layer_err_p_values = calc_p_value(listnet_mq2008_three_layer_errs, max_weighted_KL_mq2008_three_layer_errs)
print(max_weighted_KL_mq2008_three_layer_err_p_values)

[2.500161231377758, 1.671049634522606, 0.6647027721127121, 0.8564531596177498, 0.5875067963511069, 0.4770896092477601]
[0.00587127 0.00637887 0.07149401 0.00185249 0.00477275 0.0113416 ]
[2.620601133811111, 1.8083989501312059, 1.46218815603183, 1.3487385050695728, 1.3240385515111814, 1.3126240803456375]
[0.00941606 0.00542934 0.00259284 0.00315799 0.00348392 0.00358951]


#### Alpha Divergence + Entropy Regularization MQ2008 Three-Layer Network

In [19]:
file_path = 'resources/mq2008-threelayer-alpha-entropy.txt'
alpha_entropy_mq2008_three_layer_aggregate = read_file_segment(
    file_path, "Dataset (three_layer) MQ2008 NDCG", None)
alpha_entropy_mq2008_three_layer_ndcg_data = read_file_segment(
    alpha_entropy_mq2008_three_layer_aggregate, "ObjFunc", "Dataset (three_layer) MQ2008 ERRs")
alpha_entropy_mq2008_three_layer_err_data = read_file_segment(
    alpha_entropy_mq2008_three_layer_aggregate, "Dataset (three_layer) MQ2008 ERRs", None)
alpha_entropy_mq2008_three_layer_err_data = read_file_segment(
    alpha_entropy_mq2008_three_layer_err_data, "ObjFunc", None)
alpha_entropy_mq2008_three_layer_ndcg_df = read_dataframe("".join(alpha_entropy_mq2008_three_layer_ndcg_data))
alpha_entropy_mq2008_three_layer_ndcg_df.columns = ['alpha', 'lambda', 'nDCG@1', 'nDCG@3', 'nDCG@5', 'nDCG@10', 'nDCG@20', 'nDCG@50']
alpha_entropy_mq2008_three_layer_err_df = read_dataframe("".join(alpha_entropy_mq2008_three_layer_err_data))
alpha_entropy_mq2008_three_layer_err_df.columns = ['alpha', 'lambda', 'ERR@1', 'ERR@3', 'ERR@5', 'ERR@10', 'ERR@20', 'ERR@50']

In [20]:
ndcg_df_sorted = alpha_entropy_mq2008_three_layer_ndcg_df.sort_values(by='nDCG@1', ascending=False)
print(ndcg_df_sorted[:5])
err_df_sorted = alpha_entropy_mq2008_three_layer_err_df.sort_values(by='ERR@1', ascending=False)
print(err_df_sorted[:5])

best_alpha_mq2008_three_layer = 1.3
best_lambda_mq2008_three_layer = 0.7

    alpha  lambda  nDCG@1  nDCG@3  nDCG@5  nDCG@10  nDCG@20  nDCG@50
11    0.5     0.7  0.4808  0.4908  0.5349   0.6115   0.6681   0.6997
21    1.3     0.7  0.4808  0.4920  0.5385   0.6118   0.6678   0.6984
41    1.5     0.7  0.4800  0.4910  0.5366   0.6102   0.6667   0.6976
31    1.4     0.7  0.4780  0.4905  0.5346   0.6110   0.6666   0.6975
30    1.4     0.1  0.4766  0.4866  0.5324   0.6082   0.6649   0.6963
    alpha  lambda   ERR@1   ERR@3   ERR@5  ERR@10  ERR@20  ERR@50
41    1.5     0.7  0.2900  0.3892  0.4175  0.4314  0.4346  0.4353
21    1.3     0.7  0.2893  0.3892  0.4174  0.4309  0.4342  0.4349
11    0.5     0.7  0.2890  0.3882  0.4168  0.4309  0.4342  0.4348
31    1.4     0.7  0.2883  0.3887  0.4162  0.4304  0.4336  0.4342
62    1.7     1.4  0.2880  0.3905  0.4181  0.4317  0.4351  0.4357


In [21]:
max_alpha_lambda_aggregate = read_file_segment(
    'resources/mq2008-threelayer-alpha-entropy.txt',
    'Entropy regulated[0.7] alpha-Divergence[1.3] (three_layer) MQ2008 [10] runs',
    'Entropy regulated[1.4] alpha-Divergence[1.3] (three_layer) MQ2008 [10] runs')

max_alpha_lambda_mq2008_three_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_lambda_mq2008_three_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_lambda_mq2008_three_layer_ndcgs)
print(max_alpha_lambda_mq2008_three_layer_errs)

        1       2       3       4       5       6
0  0.4739  0.4907  0.5280  0.6070  0.6649  0.6968
1  0.4831  0.4990  0.5466  0.6113  0.6688  0.6997
2  0.4664  0.4897  0.5400  0.6095  0.6663  0.6960
3  0.4633  0.4893  0.5374  0.6116  0.6673  0.6964
4  0.4779  0.4877  0.5386  0.6129  0.6654  0.6973
5  0.4881  0.4948  0.5441  0.6140  0.6701  0.7007
6  0.4963  0.4998  0.5387  0.6124  0.6704  0.7012
7  0.4965  0.4923  0.5360  0.6131  0.6687  0.7005
8  0.4820  0.4887  0.5356  0.6105  0.6655  0.6961
9  0.4805  0.4880  0.5402  0.6159  0.6709  0.6988
        1       2       3       4       5       6
0  0.2850  0.3868  0.4124  0.4275  0.4310  0.4318
1  0.2978  0.3947  0.4223  0.4357  0.4391  0.4397
2  0.2799  0.3848  0.4133  0.4261  0.4296  0.4303
3  0.2747  0.3839  0.4119  0.4249  0.4280  0.4286
4  0.2885  0.3841  0.4138  0.4277  0.4309  0.4317
5  0.2967  0.3944  0.4227  0.4359  0.4390  0.4397
6  0.2946  0.3953  0.4220  0.4352  0.4385  0.4391
7  0.2997  0.3924  0.4213  0.4352  0.4388  0.4394


In [22]:
max_alpha_lambda_mq2008_three_layer_ndcg_gains = calc_gain(listnet_mq2008_three_layer_ndcgs, max_alpha_lambda_mq2008_three_layer_ndcgs)
print(max_alpha_lambda_mq2008_three_layer_ndcg_gains)
max_alpha_lambda_mq2008_three_layer_ndcg_p_values = calc_p_value(listnet_mq2008_three_layer_ndcgs, max_alpha_lambda_mq2008_three_layer_ndcgs)
print(max_alpha_lambda_mq2008_three_layer_ndcg_p_values)
max_alpha_lambda_mq2008_three_layer_err_gains = calc_gain(listnet_mq2008_three_layer_errs, max_alpha_lambda_mq2008_three_layer_errs)
print(max_alpha_lambda_mq2008_three_layer_err_gains)
max_alpha_lambda_mq2008_three_layer_err_p_values = calc_p_value(listnet_mq2008_three_layer_errs, max_alpha_lambda_mq2008_three_layer_errs)
print(max_alpha_lambda_mq2008_three_layer_err_p_values)

[3.3600619128490603, 1.8781189820470925, 1.6919706926504752, 1.1573691346185837, 0.8623814414305552, 0.6572594012597643]
[0.00196493 0.00249864 0.00088067 0.00010794 0.00022666 0.00135157]
[3.155417691731725, 2.162729658792643, 1.8796074793731483, 1.6104692289554488, 1.5766819500327558, 1.5718790143641148]
[0.00958147 0.0018621  0.00081945 0.00170402 0.00206047 0.00202377]


#### ListNet MQ2007 Single-layer NN results

In [23]:
listnet_mq2007_single_layer = read_file_segment(
    'resources/mq2007-onelayer-multi.txt', 
    'ListNet (simple_one_layer) MQ2007 metrics average of [10] runs',
    'ListMLE (simple_one_layer) MQ2007 metrics average of [10] runs')
listnet_mq2007_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2007_single_layer,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
listnet_mq2007_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2007_single_layer,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(listnet_mq2007_single_layer_ndcgs)
print(listnet_mq2007_single_layer_errs)

        1       2       3       4       5       6
0  0.4684  0.4712  0.4807  0.5116  0.5836  0.6949
1  0.4590  0.4677  0.4793  0.5102  0.5814  0.6935
2  0.4641  0.4698  0.4789  0.5123  0.5825  0.6942
3  0.4631  0.4684  0.4788  0.5106  0.5821  0.6939
4  0.4670  0.4716  0.4795  0.5111  0.5826  0.6948
5  0.4701  0.4722  0.4808  0.5119  0.5833  0.6949
6  0.4670  0.4695  0.4807  0.5111  0.5827  0.6943
7  0.4627  0.4715  0.4792  0.5112  0.5826  0.6941
8  0.4684  0.4716  0.4816  0.5120  0.5832  0.6950
9  0.4595  0.4694  0.4783  0.5099  0.5818  0.6935
        1       2       3       4       5       6
0  0.2199  0.3127  0.3397  0.3588  0.3667  0.3692
1  0.2152  0.3096  0.3366  0.3556  0.3635  0.3660
2  0.2177  0.3118  0.3383  0.3578  0.3655  0.3680
3  0.2180  0.3112  0.3384  0.3576  0.3654  0.3680
4  0.2188  0.3123  0.3388  0.3580  0.3659  0.3685
5  0.2215  0.3134  0.3403  0.3595  0.3673  0.3699
6  0.2192  0.3115  0.3391  0.3578  0.3657  0.3683
7  0.2169  0.3116  0.3379  0.3571  0.3649  0.3675


#### Alpha-divergence MQ2007 Single-layer Results

In [24]:
multi_mq2007_single_layer_aggregate = read_file_segment(
    'resources/mq2007-onelayer-multi.txt', 
    'Dataset (simple_one_layer) MQ2007 NDCG',
    None)
alpha_mq2007_single_layer_ndcgs = read_file_segment(
    multi_mq2007_single_layer_aggregate,
    "ListMLE", "Wgt"
    )
alpha_mq2007_single_layer_errs = read_file_segment(
    multi_mq2007_single_layer_aggregate,
    "Dataset (simple_one_layer) MQ2007 ERRs", "Wgt"
    )
alpha_mq2007_single_layer_errs = read_file_segment(
    alpha_mq2007_single_layer_errs,
    "ListMLE", "Wgt"
    )
alpha_mq2007_single_layer_ndcgs = read_dataframe("".join(alpha_mq2007_single_layer_ndcgs))
alpha_mq2007_single_layer_errs = read_dataframe("".join(alpha_mq2007_single_layer_errs))

max_alpha_loc = alpha_mq2007_single_layer_ndcgs[1].idxmax()
max_alpha_mq2007_single_layer_val = alpha_mq2007_single_layer_ndcgs.loc[max_alpha_loc, 0]
max_alpha_ndcgs = alpha_mq2007_single_layer_ndcgs.loc[max_alpha_loc]
max_alpha_errs = alpha_mq2007_single_layer_errs.loc[max_alpha_loc]
print(alpha_mq2007_single_layer_ndcgs)
print(max_alpha_ndcgs)
print(alpha_mq2007_single_layer_errs)
print(max_alpha_errs)
print('max_alpha:', max_alpha_mq2007_single_layer_val)

max_alpha_mq2007_single_layer_ndcg_err = read_file_segment(
    'resources/mq2007-onelayer-multi.txt', 
    'Alpha[1.6] Divergence (simple_one_layer) MQ2007 [10] runs',
    'Alpha[1.7] Divergence (simple_one_layer) MQ2007 [10] runs')
max_alpha_mq2007_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2007_single_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_mq2007_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2007_single_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_mq2007_single_layer_ndcgs)
print(max_alpha_mq2007_single_layer_errs)

          0       1       2       3       4       5       6
0  Alph-0.5  0.4674  0.4705  0.4795  0.5107  0.5821  0.6944
1   Alph0.5  0.4663  0.4698  0.4801  0.5118  0.5828  0.6943
2   Alph1.3  0.4747  0.4719  0.4810  0.5137  0.5837  0.6946
3   Alph1.4  0.4738  0.4713  0.4805  0.5137  0.5836  0.6946
4   Alph1.5  0.4736  0.4710  0.4799  0.5132  0.5834  0.6943
5   Alph1.6  0.4752  0.4715  0.4802  0.5137  0.5837  0.6946
6   Alph1.7  0.4745  0.4701  0.4791  0.5134  0.5830  0.6940
7   Alph2.0  0.4737  0.4694  0.4781  0.5121  0.5817  0.6932
8   Alph5.0  0.4400  0.4474  0.4537  0.4842  0.5548  0.6755
0    Alph1.6
1     0.4752
2     0.4715
3     0.4802
4     0.5137
5     0.5837
6     0.6946
Name: 5, dtype: object
          0       1       2       3       4       5       6
0  Alph-0.5  0.2205  0.3132  0.3397  0.3590  0.3667  0.3693
1   Alph0.5  0.2187  0.3118  0.3387  0.3578  0.3656  0.3682
2   Alph1.3  0.2223  0.3138  0.3406  0.3598  0.3675  0.3700
3   Alph1.4  0.2218  0.3137  0.3404  0.3596  0

In [25]:
max_alpha_mq2007_single_layer_ndcg_gains = calc_gain(listnet_mq2007_single_layer_ndcgs, max_alpha_mq2007_single_layer_ndcgs)
print(max_alpha_mq2007_single_layer_ndcg_gains)
max_alpha_mq2007_single_layer_ndcg_p_values = calc_p_value(listnet_mq2007_single_layer_ndcgs, max_alpha_mq2007_single_layer_ndcgs)
print(max_alpha_mq2007_single_layer_ndcg_p_values)
max_alpha_mq2007_single_layer_err_gains = calc_gain(listnet_mq2007_single_layer_errs, max_alpha_mq2007_single_layer_errs)
print(max_alpha_mq2007_single_layer_err_gains)
max_alpha_mq2007_single_layer_err_p_values = calc_p_value(listnet_mq2007_single_layer_errs, max_alpha_mq2007_single_layer_errs)
print(max_alpha_mq2007_single_layer_err_p_values)

[2.20678381691865, 0.25728805630568136, 0.08337154529159438, 0.49883604921850033, 0.1956812798242162, 0.03600697094957187]
[2.51676389e-06 4.49127892e-02 2.44782171e-01 1.50875863e-07
 2.71654881e-04 1.52864245e-01]
[2.482821804855681, 1.0813065520117848, 0.7796810395747124, 0.8303279375995898, 0.7797110965200316, 0.7715713975222651]
[1.18676138e-06 3.50016701e-07 2.32031605e-05 4.58064315e-06
 6.99746971e-06 7.10020930e-06]


#### Weighted-KL Divergence MQ2007 Single-layer Results

In [26]:
weighted_KL_mq2007_single_layer_ndcgs = read_file_segment(
    multi_mq2007_single_layer_aggregate,
    "Alph5.0", "Dataset"
    )
weighted_KL_mq2007_single_layer_errs = read_file_segment(
    multi_mq2007_single_layer_aggregate,
    "Dataset (simple_one_layer) MQ2007 ERRs", None
    )
weighted_KL_mq2007_single_layer_errs = read_file_segment(
    weighted_KL_mq2007_single_layer_errs,
    "Alph5.0", None
    )
weighted_KL_mq2007_single_layer_ndcgs = read_dataframe("".join(weighted_KL_mq2007_single_layer_ndcgs))
weighted_KL_mq2007_single_layer_errs = read_dataframe("".join(weighted_KL_mq2007_single_layer_errs))

max_lambda_loc = weighted_KL_mq2007_single_layer_ndcgs[1].idxmax()
max_lambda_mq2007_single_layer_val = weighted_KL_mq2007_single_layer_ndcgs.loc[max_lambda_loc, 0]
max_lambda_ndcgs = weighted_KL_mq2007_single_layer_ndcgs.loc[max_lambda_loc]
max_lambda_errs = weighted_KL_mq2007_single_layer_errs.loc[max_lambda_loc]
print(weighted_KL_mq2007_single_layer_ndcgs)
print(max_lambda_ndcgs)
print(weighted_KL_mq2007_single_layer_errs)
print(max_lambda_errs)
print('max_lambdaa:', max_lambda_mq2007_single_layer_val)

max_weighted_KL_mq2007_single_layer_ndcg_err = read_file_segment(
    'resources/mq2007-onelayer-multi.txt', 
    'Lambda[2.0] KL Divergence (simple_one_layer) MQ2007 [10] runs',
    'Lambda[4.0] KL Divergence (simple_one_layer) MQ2007 [10] runs')
max_weighted_KL_mq2007_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2007_single_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_weighted_KL_mq2007_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2007_single_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_weighted_KL_mq2007_single_layer_ndcgs)
print(max_weighted_KL_mq2007_single_layer_errs)

         0       1       2       3       4       5       6
0   Wgt0.1  0.4710  0.4707  0.4808  0.5135  0.5836  0.6944
1   Wgt0.7  0.4736  0.4713  0.4799  0.5139  0.5838  0.6945
2   Wgt1.4  0.4773  0.4741  0.4812  0.5155  0.5851  0.6958
3   Wgt2.0  0.4780  0.4747  0.4806  0.5151  0.5852  0.6957
4   Wgt4.0  0.4769  0.4739  0.4791  0.5145  0.5847  0.6951
5   Wgt5.0  0.4725  0.4732  0.4787  0.5140  0.5836  0.6945
6   Wgt6.0  0.4754  0.4734  0.4793  0.5148  0.5843  0.6953
7   Wgt8.0  0.4730  0.4733  0.4791  0.5144  0.5842  0.6948
8  Wgt10.0  0.4667  0.4697  0.4771  0.5127  0.5827  0.6934
9  Wgt15.0  0.4682  0.4702  0.4776  0.5130  0.5828  0.6939
0    Wgt2.0
1     0.478
2    0.4747
3    0.4806
4    0.5151
5    0.5852
6    0.6957
Name: 3, dtype: object
         0       1       2       3       4       5       6
0   Wgt0.1  0.2215  0.3132  0.3403  0.3595  0.3672  0.3697
1   Wgt0.7  0.2236  0.3149  0.3411  0.3607  0.3683  0.3709
2   Wgt1.4  0.2243  0.3156  0.3415  0.3612  0.3688  0.3714
3   Wgt2

In [27]:
max_weighted_KL_mq2007_single_layer_ndcg_gains = calc_gain(listnet_mq2007_single_layer_ndcgs, max_weighted_KL_mq2007_single_layer_ndcgs)
print(max_weighted_KL_mq2007_single_layer_ndcg_gains)
max_weighted_KL_mq2007_single_layer_ndcg_p_values = calc_p_value(listnet_mq2007_single_layer_ndcgs, max_weighted_KL_mq2007_single_layer_ndcgs)
print(max_weighted_KL_mq2007_single_layer_ndcg_p_values)
max_weighted_KL_mq2007_single_layer_err_gains = calc_gain(listnet_mq2007_single_layer_errs, max_weighted_KL_mq2007_single_layer_errs)
print(max_weighted_KL_mq2007_single_layer_err_gains)
max_weighted_KL_mq2007_single_layer_err_p_values = calc_p_value(listnet_mq2007_single_layer_errs, max_weighted_KL_mq2007_single_layer_errs)
print(max_weighted_KL_mq2007_single_layer_err_p_values)

[2.8154775987783083, 0.9398456271662123, 0.16882737921548063, 0.7551008431307509, 0.4480071406501922, 0.20451959499358743]
[7.27277939e-08 7.37055196e-06 9.63731093e-02 9.91637653e-09
 1.72409051e-07 7.41377341e-05]
[2.780577187356857, 1.3476224090354993, 0.7974010632014118, 0.9365651821409501, 0.8918800612825677, 0.8829602260378261]
[3.47072512e-07 1.17284515e-06 6.48847968e-05 9.05002853e-06
 1.10259712e-05 1.06980768e-05]


#### Alpha Divergence + Entropy Regularization MQ2007 Single-Layer Network

In [28]:
file_path = 'resources/mq2007-onelayer-alpha-entropy.txt'
alpha_entropy_mq2007_single_layer_aggregate = read_file_segment(
    file_path, "Dataset (simple_one_layer) MQ2007 NDCGs", None)
alpha_entropy_mq2007_single_layer_ndcg_data = read_file_segment(
    alpha_entropy_mq2007_single_layer_aggregate, "ObjFunc", "Dataset (simple_one_layer) MQ2007 ERRs")
alpha_entropy_mq2007_single_layer_err_data = read_file_segment(
    alpha_entropy_mq2007_single_layer_aggregate, "Dataset (simple_one_layer) MQ2007 ERRs", None)
alpha_entropy_mq2007_single_layer_err_data = read_file_segment(
    alpha_entropy_mq2007_single_layer_err_data, "ObjFunc", None)
alpha_entropy_mq2007_single_layer_ndcg_df = read_dataframe("".join(alpha_entropy_mq2007_single_layer_ndcg_data))
alpha_entropy_mq2007_single_layer_ndcg_df.columns = ['alpha', 'lambda', 'nDCG@1', 'nDCG@3', 'nDCG@5', 'nDCG@10', 'nDCG@20', 'nDCG@50']
alpha_entropy_mq2007_single_layer_err_df = read_dataframe("".join(alpha_entropy_mq2007_single_layer_err_data))
alpha_entropy_mq2007_single_layer_err_df.columns = ['alpha', 'lambda', 'ERR@1', 'ERR@3', 'ERR@5', 'ERR@10', 'ERR@20', 'ERR@50']

In [29]:
ndcg_df_sorted = alpha_entropy_mq2007_single_layer_ndcg_df.sort_values(by='nDCG@1', ascending=False)
print(ndcg_df_sorted[:5])
err_df_sorted = alpha_entropy_mq2007_single_layer_err_df.sort_values(by='ERR@1', ascending=False)
print(err_df_sorted[:5])

best_alpha_mq2007_single_layer = 1.4
best_lambda_mq2007_single_layer = 4.0

    alpha  lambda  nDCG@1  nDCG@3  nDCG@5  nDCG@10  nDCG@20  nDCG@50
34    1.4     4.0  0.4789  0.4745  0.4804   0.5147   0.5843   0.6953
32    1.4     1.4  0.4787  0.4731  0.4794   0.5143   0.5843   0.6951
23    1.3     2.0  0.4785  0.4747  0.4800   0.5150   0.5845   0.6954
33    1.4     2.0  0.4783  0.4743  0.4795   0.5149   0.5844   0.6953
64    1.7     4.0  0.4780  0.4745  0.4787   0.5134   0.5833   0.6946
    alpha  lambda   ERR@1   ERR@3   ERR@5  ERR@10  ERR@20  ERR@50
34    1.4     4.0  0.2246  0.3159  0.3414  0.3612  0.3688  0.3714
23    1.3     2.0  0.2246  0.3163  0.3414  0.3613  0.3690  0.3716
33    1.4     2.0  0.2245  0.3160  0.3411  0.3612  0.3688  0.3714
32    1.4     1.4  0.2244  0.3154  0.3409  0.3608  0.3685  0.3711
22    1.3     1.4  0.2244  0.3157  0.3413  0.3611  0.3687  0.3713


In [30]:
max_alpha_lambda_aggregate = read_file_segment(
    'resources/mq2007-onelayer-alpha-entropy.txt',
    'Entropy regulated[4.0] alpha-Divergence[1.4] (simple_one_layer) MQ2007 [10] runs',
    'Entropy regulated[5.0] alpha-Divergence[1.4] (simple_one_layer) MQ2007 [10] runs')

max_alpha_lambda_mq2007_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_lambda_mq2007_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_lambda_mq2007_single_layer_ndcgs)
print(max_alpha_lambda_mq2007_single_layer_errs)

        1       2       3       4       5       6
0  0.4785  0.4733  0.4806  0.5140  0.5822  0.6946
1  0.4778  0.4752  0.4805  0.5175  0.5856  0.6962
2  0.4726  0.4703  0.4773  0.5125  0.5828  0.6938
3  0.4824  0.4750  0.4812  0.5141  0.5844  0.6955
4  0.4820  0.4760  0.4819  0.5158  0.5847  0.6962
5  0.4757  0.4760  0.4817  0.5155  0.5847  0.6954
6  0.4806  0.4751  0.4820  0.5142  0.5842  0.6951
7  0.4804  0.4755  0.4814  0.5164  0.5859  0.6966
8  0.4792  0.4731  0.4766  0.5133  0.5840  0.6949
9  0.4798  0.4753  0.4805  0.5137  0.5843  0.6952
        1       2       3       4       5       6
0  0.2226  0.3142  0.3402  0.3597  0.3672  0.3699
1  0.2238  0.3151  0.3410  0.3612  0.3687  0.3713
2  0.2220  0.3135  0.3389  0.3592  0.3667  0.3693
3  0.2269  0.3172  0.3427  0.3620  0.3699  0.3724
4  0.2255  0.3167  0.3424  0.3622  0.3698  0.3724
5  0.2246  0.3177  0.3429  0.3623  0.3699  0.3725
6  0.2276  0.3184  0.3441  0.3636  0.3712  0.3738
7  0.2241  0.3150  0.3411  0.3605  0.3683  0.3708


In [31]:
max_alpha_lambda_mq2007_single_layer_ndcg_gains = calc_gain(listnet_mq2007_single_layer_ndcgs, max_alpha_lambda_mq2007_single_layer_ndcgs)
print(max_alpha_lambda_mq2007_single_layer_ndcg_gains)
max_alpha_lambda_mq2007_single_layer_ndcg_p_values = calc_p_value(listnet_mq2007_single_layer_ndcgs, max_alpha_lambda_mq2007_single_layer_ndcgs)
print(max_alpha_lambda_mq2007_single_layer_ndcg_p_values)
max_alpha_lambda_mq2007_single_layer_err_gains = calc_gain(listnet_mq2007_single_layer_errs, max_alpha_lambda_mq2007_single_layer_errs)
print(max_alpha_lambda_mq2007_single_layer_err_gains)
max_alpha_lambda_mq2007_single_layer_err_p_values = calc_p_value(listnet_mq2007_single_layer_errs, max_alpha_lambda_mq2007_single_layer_errs)
print(max_alpha_lambda_mq2007_single_layer_err_p_values)

[3.004753403738193, 0.8909396329924029, 0.12297302930511703, 0.6866331501007261, 0.29180541728171455, 0.14978899915024138]
[3.17716562e-08 1.26428074e-05 2.03303381e-01 1.02206412e-05
 4.89423172e-04 2.54047165e-03]
[2.890517636280327, 1.3636655329525866, 0.829887773183708, 0.9812966535268006, 0.911030860144467, 0.9128450336883367]
[2.52897504e-07 1.42455401e-06 1.90882001e-04 4.23889657e-06
 9.48766018e-06 7.77864687e-06]


#### ListNet MQ2008 Single-layer NN results

In [32]:
listnet_mq2008_single_layer = read_file_segment(
    'resources/mq2008-onelayer-multi.txt', 
    'ListNet (simple_one_layer) MQ2008 metrics average of [10] runs',
    'ListMLE (simple_one_layer) MQ2008 metrics average of [10] runs')
listnet_mq2008_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2008_single_layer,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
listnet_mq2008_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        listnet_mq2008_single_layer,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(listnet_mq2008_single_layer_ndcgs)
print(listnet_mq2008_single_layer_errs)

        1       2       3       4       5       6
0  0.4535  0.4786  0.5202  0.6035  0.6616  0.6911
1  0.4350  0.4647  0.5076  0.5926  0.6534  0.6854
2  0.4494  0.4852  0.5209  0.6054  0.6641  0.6953
3  0.4464  0.4819  0.5217  0.6027  0.6619  0.6934
4  0.4455  0.4716  0.5120  0.5975  0.6559  0.6880
5  0.4465  0.4832  0.5192  0.6029  0.6628  0.6926
6  0.4386  0.4720  0.5081  0.5964  0.6561  0.6871
7  0.4392  0.4698  0.5117  0.5957  0.6569  0.6874
8  0.4508  0.4807  0.5159  0.6001  0.6616  0.6918
9  0.4446  0.4700  0.5147  0.6017  0.6584  0.6892
        1       2       3       4       5       6
0  0.2705  0.3739  0.4002  0.4166  0.4200  0.4206
1  0.2604  0.3627  0.3909  0.4074  0.4109  0.4116
2  0.2695  0.3757  0.4019  0.4179  0.4212  0.4219
3  0.2672  0.3732  0.4000  0.4158  0.4193  0.4201
4  0.2665  0.3675  0.3948  0.4114  0.4150  0.4158
5  0.2673  0.3756  0.4006  0.4167  0.4203  0.4209
6  0.2628  0.3673  0.3932  0.4098  0.4136  0.4143
7  0.2618  0.3676  0.3949  0.4107  0.4148  0.4154


#### Alpha-divergence MQ2008 Single-layer Results

In [33]:
multi_mq2008_single_layer_aggregate = read_file_segment(
    'resources/mq2008-onelayer-multi.txt', 
    'Dataset (simple_one_layer) MQ2008 NDCG',
    None)
alpha_mq2008_single_layer_ndcgs = read_file_segment(
    multi_mq2008_single_layer_aggregate,
    "ListMLE", "Wgt"
    )
alpha_mq2008_single_layer_errs = read_file_segment(
    multi_mq2008_single_layer_aggregate,
    "Dataset (simple_one_layer) MQ2008 ERRs", "Wgt"
    )
alpha_mq2008_single_layer_errs = read_file_segment(
    alpha_mq2008_single_layer_errs,
    "ListMLE", "Wgt"
    )
alpha_mq2008_single_layer_ndcgs = read_dataframe("".join(alpha_mq2008_single_layer_ndcgs))
alpha_mq2008_single_layer_errs = read_dataframe("".join(alpha_mq2008_single_layer_errs))

max_alpha_loc = alpha_mq2008_single_layer_ndcgs[1].idxmax()
max_alpha_mq2008_single_layer_val = alpha_mq2008_single_layer_ndcgs.loc[max_alpha_loc, 0]
max_alpha_ndcgs = alpha_mq2008_single_layer_ndcgs.loc[max_alpha_loc]
max_alpha_errs = alpha_mq2008_single_layer_errs.loc[max_alpha_loc]
print(alpha_mq2008_single_layer_ndcgs)
print(max_alpha_ndcgs)
print(alpha_mq2008_single_layer_errs)
print(max_alpha_errs)
print('max_alpha:', max_alpha_mq2008_single_layer_val)

max_alpha_mq2008_single_layer_ndcg_err = read_file_segment(
    'resources/mq2008-onelayer-multi.txt', 
    'Alpha[1.5] Divergence (simple_one_layer) MQ2008 [10] runs',
    'Alpha[1.6] Divergence (simple_one_layer) MQ2008 [10] runs')
max_alpha_mq2008_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2008_single_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_mq2008_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_mq2008_single_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_mq2008_single_layer_ndcgs)
print(max_alpha_mq2008_single_layer_errs)

          0       1       2       3       4       5       6
0  Alph-0.5  0.4445  0.4757  0.5133  0.6000  0.6596  0.6904
1   Alph0.5  0.4399  0.4671  0.5142  0.5977  0.6562  0.6877
2   Alph1.3  0.4534  0.4749  0.5196  0.6000  0.6600  0.6911
3   Alph1.4  0.4512  0.4742  0.5198  0.6015  0.6603  0.6915
4   Alph1.5  0.4600  0.4752  0.5214  0.6025  0.6621  0.6927
5   Alph1.6  0.4587  0.4770  0.5225  0.6036  0.6621  0.6932
6   Alph1.7  0.4538  0.4805  0.5234  0.6044  0.6623  0.6934
7   Alph2.0  0.4491  0.4784  0.5197  0.6018  0.6606  0.6911
8   Alph5.0  0.4330  0.4577  0.5001  0.5881  0.6469  0.6773
0    Alph1.5
1       0.46
2     0.4752
3     0.5214
4     0.6025
5     0.6621
6     0.6927
Name: 4, dtype: object
          0       1       2       3       4       5       6
0  Alph-0.5  0.2654  0.3705  0.3967  0.4130  0.4167  0.4173
1   Alph0.5  0.2649  0.3677  0.3960  0.4119  0.4154  0.4161
2   Alph1.3  0.2722  0.3751  0.4029  0.4182  0.4219  0.4226
3   Alph1.4  0.2716  0.3737  0.4023  0.4179  0

In [34]:
max_alpha_mq2008_single_layer_ndcg_gains = calc_gain(listnet_mq2008_single_layer_ndcgs, max_alpha_mq2008_single_layer_ndcgs)
print(max_alpha_mq2008_single_layer_ndcg_gains)
max_alpha_mq2008_single_layer_ndcg_p_values = calc_p_value(listnet_mq2008_single_layer_ndcgs, max_alpha_mq2008_single_layer_ndcgs)
print(max_alpha_mq2008_single_layer_ndcg_p_values)
max_alpha_mq2008_single_layer_err_gains = calc_gain(listnet_mq2008_single_layer_errs, max_alpha_mq2008_single_layer_errs)
print(max_alpha_mq2008_single_layer_err_gains)
max_alpha_mq2008_single_layer_err_p_values = calc_p_value(listnet_mq2008_single_layer_errs, max_alpha_mq2008_single_layer_errs)
print(max_alpha_mq2008_single_layer_err_p_values)

[3.373412743004812, -0.11350022069484707, 1.1995341614906758, 0.4334416937567617, 0.4231953524352631, 0.36369959283035425]
[5.06105852e-05 4.36671678e-01 7.38006083e-03 6.64136500e-02
 2.95086644e-02 4.04018558e-02]
[4.445614562960712, 1.9468070743890744, 2.304321547295267, 2.0856520687152096, 2.0269135743241384, 2.028353848364407]
[1.93730304e-07 5.16714571e-04 6.01035145e-06 5.70143643e-06
 5.68554059e-06 5.22822570e-06]


#### Weighted-KL Divergence MQ2008 Single-layer Results

In [35]:
weighted_KL_mq2008_single_layer_ndcgs = read_file_segment(
    multi_mq2008_single_layer_aggregate,
    "Alph5.0", "Dataset"
    )
weighted_KL_mq2008_single_layer_errs = read_file_segment(
    multi_mq2008_single_layer_aggregate,
    "Dataset (simple_one_layer) MQ2008 ERRs", None
    )
weighted_KL_mq2008_single_layer_errs = read_file_segment(
    weighted_KL_mq2008_single_layer_errs,
    "Alph5.0", None
    )
weighted_KL_mq2008_single_layer_ndcgs = read_dataframe("".join(weighted_KL_mq2008_single_layer_ndcgs))
weighted_KL_mq2008_single_layer_errs = read_dataframe("".join(weighted_KL_mq2008_single_layer_errs))

max_lambda_loc = weighted_KL_mq2008_single_layer_ndcgs[1].idxmax()
max_lambda_mq2008_single_layer_val = weighted_KL_mq2008_single_layer_ndcgs.loc[max_lambda_loc, 0]
max_lambda_ndcgs = weighted_KL_mq2008_single_layer_ndcgs.loc[max_lambda_loc]
max_lambda_errs = weighted_KL_mq2008_single_layer_errs.loc[max_lambda_loc]
print(weighted_KL_mq2008_single_layer_ndcgs)
print(max_lambda_ndcgs)
print(weighted_KL_mq2008_single_layer_errs)
print(max_lambda_errs)
print('max_lambdaa:', max_lambda_mq2008_single_layer_val)

max_weighted_KL_mq2008_single_layer_ndcg_err = read_file_segment(
    'resources/mq2008-onelayer-multi.txt', 
    'Lambda[5.0] KL Divergence (simple_one_layer) MQ2008 [10] runs',
    'Lambda[6.0] KL Divergence (simple_one_layer) MQ2008 [10] runs')
max_weighted_KL_mq2008_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2008_single_layer_ndcg_err,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_weighted_KL_mq2008_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_weighted_KL_mq2008_single_layer_ndcg_err,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_weighted_KL_mq2008_single_layer_ndcgs)
print(max_weighted_KL_mq2008_single_layer_errs)

         0       1       2       3       4       5       6
0   Wgt0.1  0.4414  0.4721  0.5163  0.5993  0.6569  0.6892
1   Wgt0.7  0.4469  0.4737  0.5211  0.6022  0.6602  0.6910
2   Wgt1.4  0.4463  0.4781  0.5238  0.6038  0.6612  0.6919
3   Wgt2.0  0.4482  0.4763  0.5243  0.6029  0.6601  0.6907
4   Wgt4.0  0.4536  0.4839  0.5298  0.6052  0.6619  0.6927
5   Wgt5.0  0.4614  0.4880  0.5304  0.6060  0.6635  0.6944
6   Wgt6.0  0.4597  0.4859  0.5294  0.6042  0.6631  0.6936
7   Wgt8.0  0.4540  0.4814  0.5285  0.6033  0.6619  0.6919
8  Wgt10.0  0.4551  0.4800  0.5263  0.6024  0.6612  0.6915
9  Wgt15.0  0.4535  0.4829  0.5273  0.6036  0.6616  0.6924
0    Wgt5.0
1    0.4614
2     0.488
3    0.5304
4     0.606
5    0.6635
6    0.6944
Name: 5, dtype: object
         0       1       2       3       4       5       6
0   Wgt0.1  0.2650  0.3715  0.3987  0.4145  0.4179  0.4187
1   Wgt0.7  0.2696  0.3746  0.4025  0.4181  0.4214  0.4221
2   Wgt1.4  0.2692  0.3768  0.4044  0.4196  0.4228  0.4235
3   Wgt2

In [36]:
max_weighted_KL_mq2008_single_layer_ndcg_gains = calc_gain(listnet_mq2008_single_layer_ndcgs, max_weighted_KL_mq2008_single_layer_ndcgs)
print(max_weighted_KL_mq2008_single_layer_ndcg_gains)
max_weighted_KL_mq2008_single_layer_ndcg_p_values = calc_p_value(listnet_mq2008_single_layer_ndcgs, max_weighted_KL_mq2008_single_layer_ndcgs)
print(max_weighted_KL_mq2008_single_layer_ndcg_p_values)
max_weighted_KL_mq2008_single_layer_err_gains = calc_gain(listnet_mq2008_single_layer_errs, max_weighted_KL_mq2008_single_layer_errs)
print(max_weighted_KL_mq2008_single_layer_err_gains)
max_weighted_KL_mq2008_single_layer_err_p_values = calc_p_value(listnet_mq2008_single_layer_errs, max_weighted_KL_mq2008_single_layer_errs)
print(max_weighted_KL_mq2008_single_layer_err_p_values)

[3.6947971682211125, 2.566366101267419, 2.9444875776397113, 1.0252563140785176, 0.6401019309235976, 0.6114789967107971]
[4.07914117e-05 2.82268488e-04 9.72737380e-06 1.56138144e-03
 1.06647462e-02 3.77386161e-03]
[5.461110275312162, 3.6614013770757277, 3.3267855344011283, 2.838132107428023, 2.760920146801292, 2.7563580631256337]
[1.16231449e-05 6.24245050e-07 1.53041263e-07 3.41431830e-07
 3.86182557e-07 3.89762821e-07]


#### Alpha Divergence + Entropy Regularization MQ2008 Single-Layer Network

In [37]:
file_path = 'resources/mq2008-onelayer-alpha-entropy.txt'
alpha_entropy_mq2008_single_layer_aggregate = read_file_segment(
    file_path, "Dataset (simple_one_layer) MQ2008 NDCGs", None)
alpha_entropy_mq2008_single_layer_ndcg_data = read_file_segment(
    alpha_entropy_mq2008_single_layer_aggregate, "ObjFunc", "Dataset (simple_one_layer) MQ2008 ERRs")
alpha_entropy_mq2008_single_layer_err_data = read_file_segment(
    alpha_entropy_mq2008_single_layer_aggregate, "Dataset (simple_one_layer) MQ2008 ERRs", None)
alpha_entropy_mq2008_single_layer_err_data = read_file_segment(
    alpha_entropy_mq2008_single_layer_err_data, "ObjFunc", None)
alpha_entropy_mq2008_single_layer_ndcg_df = read_dataframe("".join(alpha_entropy_mq2008_single_layer_ndcg_data))
alpha_entropy_mq2008_single_layer_ndcg_df.columns = ['alpha', 'lambda', 'nDCG@1', 'nDCG@3', 'nDCG@5', 'nDCG@10', 'nDCG@20', 'nDCG@50']
alpha_entropy_mq2008_single_layer_err_df = read_dataframe("".join(alpha_entropy_mq2008_single_layer_err_data))
alpha_entropy_mq2008_single_layer_err_df.columns = ['alpha', 'lambda', 'ERR@1', 'ERR@3', 'ERR@5', 'ERR@10', 'ERR@20', 'ERR@50']

In [38]:
ndcg_df_sorted = alpha_entropy_mq2008_single_layer_ndcg_df.sort_values(by='nDCG@1', ascending=False)
print(ndcg_df_sorted[:5])
err_df_sorted = alpha_entropy_mq2008_single_layer_err_df.sort_values(by='ERR@1', ascending=False)
print(err_df_sorted[:5])

best_alpha_mq2008_single_layer = 1.5
best_lambda_mq2008_single_layer = 5.0

    alpha  lambda  nDCG@1  nDCG@3  nDCG@5  nDCG@10  nDCG@20  nDCG@50
45    1.5     5.0  0.4614  0.4860  0.5309   0.6055   0.6638   0.6939
25    1.3     5.0  0.4613  0.4846  0.5315   0.6056   0.6639   0.6940
17    0.5     8.0  0.4611  0.4857  0.5284   0.6049   0.6638   0.6943
35    1.4     5.0  0.4609  0.4820  0.5283   0.6028   0.6629   0.6934
46    1.5     6.0  0.4599  0.4844  0.5298   0.6062   0.6636   0.6937
    alpha  lambda   ERR@1   ERR@3   ERR@5  ERR@10  ERR@20  ERR@50
25    1.3     5.0  0.2802  0.3835  0.4109  0.4252  0.4285  0.4292
17    0.5     8.0  0.2801  0.3830  0.4099  0.4244  0.4281  0.4287
45    1.5     5.0  0.2796  0.3838  0.4110  0.4254  0.4288  0.4294
28    1.3    10.0  0.2795  0.3823  0.4089  0.4236  0.4271  0.4278
36    1.4     6.0  0.2794  0.3829  0.4102  0.4245  0.4279  0.4286


In [39]:
max_alpha_lambda_aggregate = read_file_segment(
    'resources/mq2008-onelayer-alpha-entropy.txt',
    'Entropy regulated[5.0] alpha-Divergence[1.5] (simple_one_layer) MQ2008 [10] runs',
    'Entropy regulated[6.0] alpha-Divergence[1.5] (simple_one_layer) MQ2008 [10] runs')

max_alpha_lambda_mq2008_single_layer_ndcgs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'NDCG@50', 'AVG,'
        ))).drop([0], axis=1)
max_alpha_lambda_mq2008_single_layer_errs = read_dataframe(
    "".join(read_file_segment(
        max_alpha_lambda_aggregate,
        'ERR@50', 'AVG,'
        ))).drop([0], axis=1)
print(max_alpha_lambda_mq2008_single_layer_ndcgs)
print(max_alpha_lambda_mq2008_single_layer_errs)

        1       2       3       4       5       6
0  0.4689  0.4888  0.5351  0.6076  0.6660  0.6957
1  0.4563  0.4905  0.5344  0.6079  0.6660  0.6957
2  0.4564  0.4861  0.5330  0.6051  0.6648  0.6945
3  0.4639  0.4843  0.5306  0.6100  0.6651  0.6956
4  0.4619  0.4876  0.5313  0.6014  0.6625  0.6921
5  0.4713  0.4868  0.5341  0.6068  0.6643  0.6945
6  0.4570  0.4794  0.5227  0.6002  0.6600  0.6901
7  0.4573  0.4895  0.5307  0.6064  0.6645  0.6939
8  0.4580  0.4883  0.5246  0.6021  0.6607  0.6925
9  0.4634  0.4786  0.5324  0.6072  0.6641  0.6940
        1       2       3       4       5       6
0  0.2840  0.3865  0.4147  0.4283  0.4317  0.4324
1  0.2756  0.3860  0.4119  0.4260  0.4292  0.4298
2  0.2777  0.3834  0.4106  0.4245  0.4280  0.4287
3  0.2783  0.3815  0.4098  0.4243  0.4275  0.4282
4  0.2798  0.3854  0.4118  0.4258  0.4294  0.4301
5  0.2853  0.3847  0.4131  0.4271  0.4304  0.4311
6  0.2781  0.3809  0.4061  0.4221  0.4258  0.4264
7  0.2784  0.3873  0.4129  0.4277  0.4309  0.4315


In [40]:
max_alpha_lambda_mq2008_single_layer_ndcg_gains = calc_gain(listnet_mq2008_single_layer_ndcgs, max_alpha_lambda_mq2008_single_layer_ndcgs)
print(max_alpha_lambda_mq2008_single_layer_ndcg_gains)
max_alpha_lambda_mq2008_single_layer_ndcg_p_values = calc_p_value(listnet_mq2008_single_layer_ndcgs, max_alpha_lambda_mq2008_single_layer_ndcgs)
print(max_alpha_lambda_mq2008_single_layer_ndcg_p_values)
max_alpha_lambda_mq2008_single_layer_err_gains = calc_gain(listnet_mq2008_single_layer_errs, max_alpha_lambda_mq2008_single_layer_errs)
print(max_alpha_lambda_mq2008_single_layer_err_gains)
max_alpha_lambda_mq2008_single_layer_err_p_values = calc_p_value(listnet_mq2008_single_layer_errs, max_alpha_lambda_mq2008_single_layer_errs)
print(max_alpha_lambda_mq2008_single_layer_err_p_values)

[3.7060343858860305, 2.1480967694474353, 3.045419254658338, 0.9369008918896135, 0.6871236367497434, 0.5404778809789638]
[1.85883368e-06 6.13020766e-04 4.76378748e-07 1.67439715e-03
 1.85775550e-03 3.06841946e-03]
[5.160222656837654, 3.637100040502192, 3.508109197139115, 2.9179772562303583, 2.8472738612103754, 2.832990085732091]
[6.79280603e-09 2.40498350e-07 1.33271214e-08 6.95860431e-08
 6.11173061e-08 5.74640327e-08]


### ListMLE data

In [41]:
multi_mq2007_single_layer = read_file_segment(
    'resources/mq2007-onelayer-multi.txt',
    "Dataset (simple_one_layer) MQ2007 NDCGs", None)
listmle_mq2007_single_layer_ndcgs = read_file_segment(
    multi_mq2007_single_layer,
    "ListNet", "Alph"
    )
listmle_mq2007_single_layer_errs = read_file_segment(
    multi_mq2007_single_layer,
    "Dataset (simple_one_layer) MQ2007 ERRs", "Alph"
    )
listmle_mq2007_single_layer_errs = read_file_segment(
    listmle_mq2007_single_layer_errs,
    "ListNet", None)
listmle_mq2007_single_layer_ndcgs = read_dataframe("".join(listmle_mq2007_single_layer_ndcgs)).drop([0], axis=1)
listmle_mq2007_single_layer_errs = read_dataframe("".join(listmle_mq2007_single_layer_errs)).drop([0], axis=1)

multi_mq2008_single_layer = read_file_segment(
    'resources/mq2008-onelayer-multi.txt',
    "Dataset (simple_one_layer) MQ2008 NDCGs", None)
listmle_mq2008_single_layer_ndcgs = read_file_segment(
    multi_mq2008_single_layer,
    "ListNet", "Alph"
    )
listmle_mq2008_single_layer_errs = read_file_segment(
    multi_mq2008_single_layer,
    "Dataset (simple_one_layer) MQ2008 ERRs", "Alph"
    )
listmle_mq2008_single_layer_errs = read_file_segment(
    listmle_mq2008_single_layer_errs,
    "ListNet", None)
listmle_mq2008_single_layer_ndcgs = read_dataframe("".join(listmle_mq2008_single_layer_ndcgs)).drop([0], axis=1)
listmle_mq2008_single_layer_errs = read_dataframe("".join(listmle_mq2008_single_layer_errs)).drop([0], axis=1)

multi_mq2007_three_layer = read_file_segment(
    'resources/mq2007-threelayer-multi.txt',
    "Dataset (three_layer) MQ2007 NDCGs", None)
listmle_mq2007_three_layer_ndcgs = read_file_segment(
    multi_mq2007_three_layer,
    "ListNet", "Alph"
    )
listmle_mq2007_three_layer_errs = read_file_segment(
    multi_mq2007_three_layer,
    "Dataset (three_layer) MQ2007 ERRs", "Alph"
    )
listmle_mq2007_three_layer_errs = read_file_segment(
    listmle_mq2007_three_layer_errs,
    "ListNet", None)
listmle_mq2007_three_layer_ndcgs = read_dataframe("".join(listmle_mq2007_three_layer_ndcgs)).drop([0], axis=1)
listmle_mq2007_three_layer_errs = read_dataframe("".join(listmle_mq2007_three_layer_errs)).drop([0], axis=1)

multi_mq2008_three_layer = read_file_segment(
    'resources/mq2008-threelayer-multi.txt',
    "Dataset (three_layer) MQ2008 NDCGs", None)
listmle_mq2008_three_layer_ndcgs = read_file_segment(
    multi_mq2008_three_layer,
    "ListNet", "Alph"
    )
listmle_mq2008_three_layer_errs = read_file_segment(
    multi_mq2008_three_layer,
    "Dataset (three_layer) MQ2008 ERRs", "Alph"
    )
listmle_mq2008_three_layer_errs = read_file_segment(
    listmle_mq2008_three_layer_errs,
    "ListNet", None)
listmle_mq2008_three_layer_ndcgs = read_dataframe("".join(listmle_mq2008_three_layer_ndcgs)).drop([0], axis=1)
listmle_mq2008_three_layer_errs = read_dataframe("".join(listmle_mq2008_three_layer_errs)).drop([0], axis=1)

### Tabular data generation

#### MQ–2007 (single-layer neural network)

In [42]:
def format_line_text(head, list_data, tail=" \\\\"):
    line_text = [f"{itm:1.4f}" for itm in list_data]
    return " & ".join([head] + line_text) + tail

def format_line_text_with_significance(head, list_data, p_values, mask, tail=" \\\\"):
    line_text = [("\\textbf{" if itm[2] else "") + f"{itm[0]:1.4f}" 
                 + ("}" if itm[2] else "")
                 + ("*" if itm[1] <= 0.05 else "") for itm in zip(list_data, p_values, mask)]
    return " & ".join([head] + line_text) + tail

listmle_data = listmle_mq2007_single_layer_ndcgs.loc[0,:].tolist()[:-2] + listmle_mq2007_single_layer_errs.loc[0,:].tolist()[:-2]
listnet_data = listnet_mq2007_single_layer_ndcgs.mean().tolist()[:-2] + listnet_mq2007_single_layer_errs.mean().tolist()[:-2]
alpha_div_data = max_alpha_mq2007_single_layer_ndcgs.mean().tolist()[:-2] + max_alpha_mq2007_single_layer_errs.mean().tolist()[:-2]
alpha_div_p_values = max_alpha_mq2007_single_layer_ndcg_p_values[:-2].tolist() + max_alpha_mq2007_single_layer_err_p_values[:-2].tolist()
weighted_KL_data = max_weighted_KL_mq2007_single_layer_ndcgs.mean().tolist()[:-2] + max_weighted_KL_mq2007_single_layer_errs.mean().tolist()[:-2]
weighted_KL_p_values = max_weighted_KL_mq2007_single_layer_ndcg_p_values[:-2].tolist() + max_weighted_KL_mq2007_single_layer_err_p_values[:-2].tolist()
alpha_weighted_KL_data = max_alpha_lambda_mq2007_single_layer_ndcgs.mean().tolist()[:-2] + max_alpha_lambda_mq2007_single_layer_errs.mean().tolist()[:-2]
alpha_weighted_KL_p_values = max_alpha_lambda_mq2007_single_layer_ndcg_p_values[:-2].tolist() + max_alpha_lambda_mq2007_single_layer_err_p_values[:-2].tolist()

mq2007_single_layer_all_data = pd.DataFrame(
    {'alpha': alpha_div_data, 'wkl': weighted_KL_data, 'al_wkl':  alpha_weighted_KL_data}).transpose()
all_masks = mq2007_single_layer_all_data.eq(mq2007_single_layer_all_data.max(axis=0), axis=1)

listmle_line = format_line_text('ListMLE', listmle_data)
listnet_line = format_line_text('ListNet', listnet_data)
alpha_div_line = format_line_text_with_significance(
    f"$\\alpha$-Div({max_alpha_mq2007_single_layer_val[4:]})" ,
    alpha_div_data, alpha_div_p_values, all_masks.iloc[0].tolist())
weighted_KL_line = format_line_text_with_significance(
    f"Ent({max_lambda_mq2007_single_layer_val[3:]})" ,
    weighted_KL_data, weighted_KL_p_values, all_masks.iloc[1].tolist())
alpha_weighted_KL_line = format_line_text_with_significance(
    f"$\\alpha$({best_alpha_mq2007_single_layer:1.1f})-Ent({best_lambda_mq2007_single_layer:1.1f})" ,
    alpha_weighted_KL_data, alpha_weighted_KL_p_values,
    all_masks.iloc[2].tolist())

print(mq2007_single_layer_all_data)
print("")

print(listmle_line)
print(listnet_line)
print(alpha_div_line)
print(weighted_KL_line)
print(alpha_weighted_KL_line)

print("")
print(f"nDCG@1 Gains: alpha - {max_alpha_mq2007_single_layer_ndcg_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2007_single_layer_ndcg_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2007_single_layer_ndcg_gains[0]:1.2f}%")
print(f" ERR@1 Gains: alpha - {max_alpha_mq2007_single_layer_err_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2007_single_layer_err_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2007_single_layer_err_gains[0]:1.2f}%")

              0        1        2        3        4        5        6        7
alpha   0.47519  0.47150  0.48018  0.51374  0.22372  0.31503  0.34124  0.36066
wkl     0.47802  0.47471  0.48059  0.51505  0.22437  0.31586  0.34130  0.36104
al_wkl  0.47890  0.47448  0.48037  0.51470  0.22461  0.31591  0.34141  0.36120

ListMLE & 0.4297 & 0.4369 & 0.4461 & 0.4836 & 0.1985 & 0.2898 & 0.3168 & 0.3373 \\
ListNet & 0.4649 & 0.4703 & 0.4798 & 0.5112 & 0.2183 & 0.3117 & 0.3386 & 0.3577 \\
$\alpha$-Div(1.6) & 0.4752* & 0.4715* & 0.4802 & 0.5137* & 0.2237* & 0.3150* & 0.3412* & 0.3607* \\
Ent(2.0) & 0.4780* & \textbf{0.4747}* & \textbf{0.4806} & \textbf{0.5151}* & 0.2244* & 0.3159* & 0.3413* & 0.3610* \\
$\alpha$(1.4)-Ent(4.0) & \textbf{0.4789}* & 0.4745* & 0.4804 & 0.5147* & \textbf{0.2246}* & \textbf{0.3159}* & \textbf{0.3414}* & \textbf{0.3612}* \\

nDCG@1 Gains: alpha - 2.21%; weighted-KL - 2.82%; alpha+weighted-KL - 3.00%
 ERR@1 Gains: alpha - 2.48%; weighted-KL - 2.78%; alpha+weighted-KL - 2.

#### MQ–2008 (single-layer neural network)

In [43]:
listmle_data = listmle_mq2008_single_layer_ndcgs.loc[0,:].tolist()[:-2] + listmle_mq2008_single_layer_errs.loc[0,:].tolist()[:-2]
listnet_data = listnet_mq2008_single_layer_ndcgs.mean().tolist()[:-2] + listnet_mq2008_single_layer_errs.mean().tolist()[:-2]
alpha_div_data = max_alpha_mq2008_single_layer_ndcgs.mean().tolist()[:-2] + max_alpha_mq2008_single_layer_errs.mean().tolist()[:-2]
alpha_div_p_values = max_alpha_mq2008_single_layer_ndcg_p_values[:-2].tolist() + max_alpha_mq2008_single_layer_err_p_values[:-2].tolist()
weighted_KL_data = max_weighted_KL_mq2008_single_layer_ndcgs.mean().tolist()[:-2] + max_weighted_KL_mq2008_single_layer_errs.mean().tolist()[:-2]
weighted_KL_p_values = max_weighted_KL_mq2008_single_layer_ndcg_p_values[:-2].tolist() + max_weighted_KL_mq2008_single_layer_err_p_values[:-2].tolist()
alpha_weighted_KL_data = max_alpha_lambda_mq2008_single_layer_ndcgs.mean().tolist()[:-2] + max_alpha_lambda_mq2008_single_layer_errs.mean().tolist()[:-2]
alpha_weighted_KL_p_values = max_alpha_lambda_mq2008_single_layer_ndcg_p_values[:-2].tolist() + max_alpha_lambda_mq2008_single_layer_err_p_values[:-2].tolist()

mq2008_single_layer_all_data = pd.DataFrame(
    {'alpha': alpha_div_data, 'wkl': weighted_KL_data, 'al_wkl':  alpha_weighted_KL_data}).transpose()
all_masks = mq2008_single_layer_all_data.eq(mq2008_single_layer_all_data.max(axis=0), axis=1)

listmle_line = format_line_text('ListMLE', listmle_data)
listnet_line = format_line_text('ListNet', listnet_data)
alpha_div_line = format_line_text_with_significance(
    f"$\\alpha$-Div({max_alpha_mq2008_single_layer_val[4:]})" ,
    alpha_div_data, alpha_div_p_values, all_masks.iloc[0].tolist())
weighted_KL_line = format_line_text_with_significance(
    f"Ent({max_lambda_mq2008_single_layer_val[3:]})" ,
    weighted_KL_data, weighted_KL_p_values, all_masks.iloc[1].tolist())
alpha_weighted_KL_line = format_line_text_with_significance(
    f"$\\alpha$({best_alpha_mq2008_single_layer:1.1f})-Ent({best_lambda_mq2008_single_layer:1.1f})" ,
    alpha_weighted_KL_data, alpha_weighted_KL_p_values,
    all_masks.iloc[2].tolist())

print(mq2008_single_layer_all_data)
print("")

print(listmle_line)
print(listnet_line)
print(alpha_div_line)
print(weighted_KL_line)
print(alpha_weighted_KL_line)

print("")
print(f"nDCG@1 Gains: alpha - {max_alpha_mq2008_single_layer_ndcg_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2008_single_layer_ndcg_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2008_single_layer_ndcg_gains[0]:1.2f}%")
print(f" ERR@1 Gains: alpha - {max_alpha_mq2008_single_layer_err_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2008_single_layer_err_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2008_single_layer_err_gains[0]:1.2f}%")

              0        1        2        3       4        5        6        7
alpha   0.45996  0.47523  0.52138  0.60245  0.2777  0.37756  0.40623  0.42192
wkl     0.46139  0.48798  0.53037  0.60600  0.2804  0.38391  0.41029  0.42503
al_wkl  0.46144  0.48599  0.53089  0.60547  0.2796  0.38382  0.41101  0.42536

ListMLE & 0.4122 & 0.4468 & 0.4895 & 0.5722 & 0.2482 & 0.3527 & 0.3791 & 0.3953 \\
ListNet & 0.4450 & 0.4758 & 0.5152 & 0.5999 & 0.2659 & 0.3704 & 0.3971 & 0.4133 \\
$\alpha$-Div(1.5) & 0.4600* & 0.4752 & 0.5214* & 0.6025 & 0.2777* & 0.3776* & 0.4062* & 0.4219* \\
Ent(5.0) & 0.4614* & \textbf{0.4880}* & 0.5304* & \textbf{0.6060}* & \textbf{0.2804}* & \textbf{0.3839}* & 0.4103* & 0.4250* \\
$\alpha$(1.5)-Ent(5.0) & \textbf{0.4614}* & 0.4860* & \textbf{0.5309}* & 0.6055* & 0.2796* & 0.3838* & \textbf{0.4110}* & \textbf{0.4254}* \\

nDCG@1 Gains: alpha - 3.37%; weighted-KL - 3.69%; alpha+weighted-KL - 3.71%
 ERR@1 Gains: alpha - 4.45%; weighted-KL - 5.46%; alpha+weighted-KL - 5.16%

#### MQ–2007 (three-layer neural network)

In [44]:
listmle_data = listmle_mq2007_three_layer_ndcgs.loc[0,:].tolist()[:-2] + listmle_mq2007_three_layer_errs.loc[0,:].tolist()[:-2]
listnet_data = listnet_mq2007_three_layer_ndcgs.mean().tolist()[:-2] + listnet_mq2007_three_layer_errs.mean().tolist()[:-2]
alpha_div_data = max_alpha_mq2007_three_layer_ndcgs.mean().tolist()[:-2] + max_alpha_mq2007_three_layer_errs.mean().tolist()[:-2]
alpha_div_p_values = max_alpha_mq2007_three_layer_ndcg_p_values[:-2].tolist() + max_alpha_mq2007_three_layer_err_p_values[:-2].tolist()
weighted_KL_data = max_weighted_KL_mq2007_three_layer_ndcgs.mean().tolist()[:-2] + max_weighted_KL_mq2007_three_layer_errs.mean().tolist()[:-2]
weighted_KL_p_values = max_weighted_KL_mq2007_three_layer_ndcg_p_values[:-2].tolist() + max_weighted_KL_mq2007_three_layer_err_p_values[:-2].tolist()
alpha_weighted_KL_data = max_alpha_lambda_mq2007_three_layer_ndcgs.mean().tolist()[:-2] + max_alpha_lambda_mq2007_three_layer_errs.mean().tolist()[:-2]
alpha_weighted_KL_p_values = max_alpha_lambda_mq2007_three_layer_ndcg_p_values[:-2].tolist() + max_alpha_lambda_mq2007_three_layer_err_p_values[:-2].tolist()

mq2007_three_layer_all_data = pd.DataFrame(
    {'alpha': alpha_div_data, 'wkl': weighted_KL_data, 'al_wkl':  alpha_weighted_KL_data}).transpose()
all_masks = mq2007_three_layer_all_data.eq(mq2007_three_layer_all_data.max(axis=0), axis=1)

listmle_line = format_line_text('ListMLE', listmle_data)
listnet_line = format_line_text('ListNet', listnet_data)
alpha_div_line = format_line_text_with_significance(
    f"$\\alpha$-Div({max_alpha_mq2007_three_layer_val[4:]})" ,
    alpha_div_data, alpha_div_p_values, all_masks.iloc[0].tolist())
weighted_KL_line = format_line_text_with_significance(
    f"Ent({max_lambda_mq2007_three_layer_val[3:]})" ,
    weighted_KL_data, weighted_KL_p_values, all_masks.iloc[1].tolist())
alpha_weighted_KL_line = format_line_text_with_significance(
    f"$\\alpha$({best_alpha_mq2007_three_layer:1.1f})-Ent({best_lambda_mq2007_three_layer:1.1f})" ,
    alpha_weighted_KL_data, alpha_weighted_KL_p_values,
    all_masks.iloc[2].tolist())

print(mq2007_three_layer_all_data)
print("")

print(listmle_line)
print(listnet_line)
print(alpha_div_line)
print(weighted_KL_line)
print(alpha_weighted_KL_line)

print("")
print(f"nDCG@1 Gains: alpha - {max_alpha_mq2007_three_layer_ndcg_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2007_three_layer_ndcg_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2007_three_layer_ndcg_gains[0]:1.2f}%")
print(f" ERR@1 Gains: alpha - {max_alpha_mq2007_three_layer_err_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2007_three_layer_err_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2007_three_layer_err_gains[0]:1.2f}%")

              0        1        2        3        4        5        6        7
alpha   0.47713  0.47539  0.48348  0.51777  0.22605  0.31726  0.34358  0.36315
wkl     0.47539  0.47397  0.48187  0.51658  0.22641  0.31678  0.34265  0.36239
al_wkl  0.47727  0.47560  0.48312  0.51693  0.22615  0.31714  0.34338  0.36281

ListMLE & 0.4452 & 0.4539 & 0.4631 & 0.4989 & 0.2121 & 0.3053 & 0.3315 & 0.3513 \\
ListNet & 0.4728 & 0.4743 & 0.4817 & 0.5167 & 0.2231 & 0.3155 & 0.3417 & 0.3613 \\
$\alpha$-Div(0.4) & 0.4771* & 0.4754 & \textbf{0.4835}* & \textbf{0.5178}* & 0.2261* & \textbf{0.3173}* & \textbf{0.3436}* & \textbf{0.3631}* \\
Ent(-0.9) & 0.4754* & 0.4740 & 0.4819 & 0.5166 & \textbf{0.2264}* & 0.3168* & 0.3427 & 0.3624* \\
$\alpha$(0.4)-Ent(-0.4) & \textbf{0.4773}* & \textbf{0.4756} & 0.4831 & 0.5169 & 0.2261* & 0.3171* & 0.3434* & 0.3628* \\

nDCG@1 Gains: alpha - 0.91%; weighted-KL - 0.54%; alpha+weighted-KL - 0.93%
 ERR@1 Gains: alpha - 1.32%; weighted-KL - 1.48%; alpha+weighted-KL - 1.36%

#### MQ–2008 (three-layer neural network)

In [45]:
listmle_data = listmle_mq2008_three_layer_ndcgs.loc[0,:].tolist()[:-2] + listmle_mq2008_three_layer_errs.loc[0,:].tolist()[:-2]
listnet_data = listnet_mq2008_three_layer_ndcgs.mean().tolist()[:-2] + listnet_mq2008_three_layer_errs.mean().tolist()[:-2]
alpha_div_data = max_alpha_mq2008_three_layer_ndcgs.mean().tolist()[:-2] + max_alpha_mq2008_three_layer_errs.mean().tolist()[:-2]
alpha_div_p_values = max_alpha_mq2008_three_layer_ndcg_p_values[:-2].tolist() + max_alpha_mq2008_three_layer_err_p_values[:-2].tolist()
weighted_KL_data = max_weighted_KL_mq2008_three_layer_ndcgs.mean().tolist()[:-2] + max_weighted_KL_mq2008_three_layer_errs.mean().tolist()[:-2]
weighted_KL_p_values = max_weighted_KL_mq2008_three_layer_ndcg_p_values[:-2].tolist() + max_weighted_KL_mq2008_three_layer_err_p_values[:-2].tolist()
alpha_weighted_KL_data = max_alpha_lambda_mq2008_three_layer_ndcgs.mean().tolist()[:-2] + max_alpha_lambda_mq2008_three_layer_errs.mean().tolist()[:-2]
alpha_weighted_KL_p_values = max_alpha_lambda_mq2008_three_layer_ndcg_p_values[:-2].tolist() + max_alpha_lambda_mq2008_three_layer_err_p_values[:-2].tolist()

mq2008_three_layer_all_data = pd.DataFrame(
    {'alpha': alpha_div_data, 'wkl': weighted_KL_data, 'al_wkl':  alpha_weighted_KL_data}).transpose()
all_masks = mq2008_three_layer_all_data.eq(mq2008_three_layer_all_data.max(axis=0), axis=1)

listmle_line = format_line_text('ListMLE', listmle_data)
listnet_line = format_line_text('ListNet', listnet_data)
alpha_div_line = format_line_text_with_significance(
    f"$\\alpha$-Div({max_alpha_mq2008_three_layer_val[4:]})" ,
    alpha_div_data, alpha_div_p_values, all_masks.iloc[0].tolist())
weighted_KL_line = format_line_text_with_significance(
    f"Ent({max_lambda_mq2008_three_layer_val[3:]})" ,
    weighted_KL_data, weighted_KL_p_values, all_masks.iloc[1].tolist())
alpha_weighted_KL_line = format_line_text_with_significance(
    f"$\\alpha$({best_alpha_mq2008_three_layer:1.1f})-Ent({best_lambda_mq2008_three_layer:1.1f})" ,
    alpha_weighted_KL_data, alpha_weighted_KL_p_values,
    all_masks.iloc[2].tolist())

print(mq2008_three_layer_all_data)
print("")

print(listmle_line)
print(listnet_line)
print(alpha_div_line)
print(weighted_KL_line)
print(alpha_weighted_KL_line)

print("")
print(f"nDCG@1 Gains: alpha - {max_alpha_mq2008_three_layer_ndcg_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2008_three_layer_ndcg_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2008_three_layer_ndcg_gains[0]:1.2f}%")
print(f" ERR@1 Gains: alpha - {max_alpha_mq2008_three_layer_err_gains[0]:1.2f}%; weighted-KL - {max_weighted_KL_mq2008_three_layer_err_gains[0]:1.2f}%; alpha+weighted-KL - {max_alpha_lambda_mq2008_three_layer_err_gains[0]:1.2f}%")

              0        1        2        3        4        5        6        7
alpha   0.47639  0.48832  0.53413  0.60990  0.28655  0.38594  0.41425  0.42841
wkl     0.47680  0.49100  0.53308  0.61000  0.28782  0.38789  0.41565  0.42982
al_wkl  0.48080  0.49200  0.53852  0.61182  0.28932  0.38924  0.41736  0.43093

ListMLE & 0.4283 & 0.4550 & 0.4936 & 0.5726 & 0.2554 & 0.3584 & 0.3835 & 0.4001 \\
ListNet & 0.4652 & 0.4829 & 0.5296 & 0.6048 & 0.2805 & 0.3810 & 0.4097 & 0.4241 \\
$\alpha$-Div(0.5) & 0.4764* & 0.4883* & 0.5341* & 0.6099* & 0.2865* & 0.3859* & 0.4143* & 0.4284* \\
Ent(0.7) & 0.4768* & 0.4910* & 0.5331 & 0.6100* & 0.2878* & 0.3879* & 0.4157* & 0.4298* \\
$\alpha$(1.3)-Ent(0.7) & \textbf{0.4808}* & \textbf{0.4920}* & \textbf{0.5385}* & \textbf{0.6118}* & \textbf{0.2893}* & \textbf{0.3892}* & \textbf{0.4174}* & \textbf{0.4309}* \\

nDCG@1 Gains: alpha - 2.41%; weighted-KL - 2.50%; alpha+weighted-KL - 3.36%
 ERR@1 Gains: alpha - 2.17%; weighted-KL - 2.62%; alpha+weighted-KL - 